# 手眼标定

小朱-学长 2024-10-19
## 手眼标定的原理

已知图像上两个点的【像素坐标】和【机械臂坐标】，就可以通过【线性插值】，建立像素坐标到机械臂坐标的映射关系。输入图像任意一点的像素坐标，hand_eye_calibration函数就能转换为机械臂坐标。让机械臂移动到图像上的同一个位置。

在xzarm_robot.py里修改hand_eye_calibration函数，填好八个数字坐标。

## 导入工具包

In [6]:
from pymycobot import MyCobot280
from pymycobot import PI_PORT, PI_BAUD
import time

import cv2,os
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## 连接机械臂

In [2]:
mc = MyCobot280(PI_PORT, PI_BAUD)

## 设置运动模式为插补

In [ ]:
mc.set_fresh_mode(0)

## 机械臂归零

In [4]:
mc.send_angles([0, 0, 0, 0, 0, 0], 40)
time.sleep(3)

## 第一步：移动至俯视姿态

俯视姿态（关节）：[2.46, -8.26, -5.53, -74.0, -0.43, 94.74]

In [ ]:
mc.get_angles()

In [5]:
mc.send_angles([2.46, -8.26, -5.53, -74.0, -0.43, 94.74], 10)
time.sleep(3)

## 第二步：在白纸靠左上角的位置，画一个标定点

## 第三步：运行xzarm_camera.py，打开摄像头实时画面

## 第四步：把白纸左上角对准画面左上角

## 第五步：白纸上边与底座、图像上边平齐，白纸下边与图像下边平齐

## 第六步：用夹子固定白纸，分别夹左上角和右下角。（把麦克风线也固定）

## 第七步：通过鼠标点选，获取白纸左上角标定点，在图像上的像素坐标

In [ ]:
# 第一个标定点的像素坐标
cali_1_pixel = [591, 435]

## 第八步：控制机械臂，移动至左上角第一个标定点

8.1机械臂断电

In [42]:
# 移动到标定点附近
mc.release_all_servos()
time.sleep(3)

8.2机械臂定点

In [43]:
angles=mc.get_angles()
time.sleep(0.1)
mc.send_angles(angles,10)
time.sleep(2)

8.3微调关节

In [ ]:
mc.get_angles()

In [ ]:
n=3         #关节轴
m=30          #角度
mc.send_angle(n,m,10)
time.sleep(1)

In [ ]:
mc.get_coords()

8.4微调坐标

In [ ]:
mc.get_coords()

In [8]:
n=6         #XYZ坐标数
m=180       #值
mc.send_coord(n, m, 20)
time.sleep(1)

8.5记下机械臂坐标

In [ ]:
# 第一个标定点的机械臂坐标
cali_1_mc = [-21.8, -197.4]

## 第九步：控制机械臂，移动至右下角第二个标定点

In [ ]:
# 机械臂归零
mc.send_angles([0, 0, 0, 0, 0, 0], 40)
time.sleep(3)

In [ ]:
# 第二个标定点的像素坐标
cali_2_pixel = [51, 71]

8.1机械臂断电

In [ ]:
# 移动到标定点附近
mc.release_all_servos()
time.sleep(3)

8.2机械臂定点

In [ ]:
angles=mc.get_angles()
time.sleep(0.1)
mc.send_angles(angles,10)
time.sleep(2)

In [ ]:
mc.get_angles()

8.3微调关节

In [ ]:
n=6#关节轴
m=120#角度

mc.send_angle(n,m,10)
time.sleep(1)

8.4微调坐标

In [ ]:
mc.get_coords()

In [ ]:
n=6         #XYZ坐标数
m=120       #值
mc.send_coord(n, m, 20)
time.sleep(1)

9.5记下机械臂坐标

In [ ]:
# 第二个标定点的机械臂坐标
cali_2_mc = [51.0, -140.6]

## 第十步：通过插值，获取图像任意像素坐标对应的机械臂坐标

In [2]:
# 整理两个标定点的坐标
cali_1_pixel = [591,435]                       # 左上角，第一个标定点的像素坐标，要手动填！
cali_1_mc = [239.6, 117.1]                  # 左上角，第一个标定点的机械臂坐标，要手动填！
cali_2_pixel = [51,71]                         # 右下角，第二个标定点的像素坐标
cali_2_mc = [51.0, -140.6]                    # 右下角，第二个标定点的机械臂坐标，要手动填！

In [9]:
# 中间指定点在图像中的像素坐标
X_pixel = 320
Y_pixel = 240

In [10]:
# 计算机械臂的Y轴（对应图像的X轴）上的坐标
Y_cali_im = [cali_2_pixel[0], cali_1_pixel[0]]  # 图像的X轴像素坐标（先小后大）
Y_cali_mc = [cali_2_mc[1], cali_1_mc[1]]  # 机械臂的Y轴坐标

Y_mc = np.interp(X_pixel, Y_cali_im, Y_cali_mc)  # 使用插值法计算机械臂Y轴的坐标

# 计算机械臂的X轴（对应图像的Y轴）上的坐标
X_cali_im = [cali_2_pixel[1], cali_1_pixel[1]]  # 图像的Y轴像素坐标（先小后大）
X_cali_mc = [cali_2_mc[0], cali_1_mc[0]]  # 机械臂的X轴坐标

X_mc = np.interp(Y_pixel, X_cali_im, X_cali_mc)  # 使用插值法计算机械臂X轴的坐标


In [ ]:
print(f"像素点({X_pixel}, {Y_pixel})对应的机械臂坐标为: ({X_mc}, {Y_mc})")

## 让机械臂移动至该点吸取

In [13]:
mc.send_coords([X_mc, Y_mc, 100, -178.24, 1.68, -134.33], 20)
time.sleep(3)

In [ ]:
# 机械臂归零
mc.send_angles([0, 0, 0, 0, 0, 0], 40)
time.sleep(3)

## 封装手眼标定函数

In [12]:
import numpy as np
def hand_eye_calibration(X_pixel=160, Y_pixel=120):
    '''
    输入目标点在图像中的像素坐标，转换为机械臂坐标
    '''
    # 计算机械臂的Y轴（对应图像的X轴）上的坐标
    Y_cali_im = [cali_2_pixel[0], cali_1_pixel[0]]  # 图像的X轴像素坐标（先小后大）
    Y_cali_mc = [cali_2_mc[1], cali_1_mc[1]]  # 机械臂的Y轴坐标
    
    Y_mc = np.interp(X_pixel, Y_cali_im, Y_cali_mc)  # 使用插值法计算机械臂Y轴的坐标
    
    # 计算机械臂的X轴（对应图像的Y轴）上的坐标
    X_cali_im = [cali_2_pixel[1], cali_1_pixel[1]]  # 图像的Y轴像素坐标（先小后大）
    X_cali_mc = [cali_2_mc[0], cali_1_mc[0]]  # 机械臂的X轴坐标
    
    X_mc = np.interp(Y_pixel, X_cali_im, X_cali_mc)  # 使用插值法计算机械臂X轴的坐标


    return X_mc, Y_mc

In [13]:
hand_eye_calibration(X_pixel=163, Y_pixel=267)

(152.55384615384614, -87.1511111111111)

In [14]:
hand_eye_calibration(X_pixel=320, Y_pixel=240)

(138.5642857142857, -12.227222222222224)

## 第十一步：验证标定效果

In [61]:
# 机械臂归零
mc.send_angles([0, 0, 0, 0, 0, 0], 40)
time.sleep(3)

In [ ]:
# 移动至俯视姿态
mc.send_angles([-62.13, 8.96, -87.71, -14.41, 2.54, -16.34], 10)
time.sleep(3)

In [62]:
# 运行`xzarm_check.py`，用鼠标点选图像中的某个点，获取像素坐标
X_pixel, Y_pixel = 380,471

In [63]:
# 手眼标定转换为机械臂坐标
X_mc, Y_mc = hand_eye_calibration(X_pixel, Y_pixel)

In [ ]:
X_mc

In [65]:
Y_mc=Y_mc-20

In [ ]:
Y_mc

In [69]:
# 控制机械臂移动到这个点，看是否准确
mc.send_coords([X_mc, Y_mc, 150, -178.24, 1.68, -134.33], 20)
time.sleep(3)